In [23]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import dash
import dash_core_components as dcc                  # 交互式组件
import dash_html_components as html                 # 代码转html
from dash.dependencies import Input, Output         # 回调
from jupyter_plotly_dash import JupyterDash
from sklearn.cluster import KMeans
import numpy as np

In [2]:
records = pd.read_csv('data-utf8-loc.csv')
df = pd.DataFrame(records)

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [4]:
le = preprocessing.LabelEncoder()

In [5]:
X1=le.fit(df['district'].unique()).transform(df['district']) #X1表示房屋所在区
X2=le.fit(df['house_type'].unique()).transform(df['house_type']) #X2表示房屋的户型

In [6]:
X3=df['area']  #X3表示房屋的面积
X4=le.fit(df['fixture'].unique()).transform(df['fixture'])

In [9]:
X5=le.fit(df['elevator_exist'].unique()).transform(df['elevator_exist']) #X5表示房屋有无电梯
X6=le.fit(df['xiaoqu'].unique()).transform(df['xiaoqu']) 
X7=le.fit(df['weizhi'].unique()).transform(df['weizhi'])

In [10]:
X=np.mat([X1,X2,X3,X4,X5,X6,X7]).T

In [11]:
Y=df['unit_price']

In [12]:
print (np.array(X).shape)

(14396, 7)


In [13]:
X

matrix([[0.000e+00, 2.200e+01, 8.900e+01, ..., 1.000e+00, 3.940e+02,
         3.000e+00],
        [0.000e+00, 4.000e+00, 3.330e+01, ..., 1.000e+00, 7.300e+02,
         3.000e+01],
        [0.000e+00, 4.500e+01, 1.210e+02, ..., 0.000e+00, 1.236e+03,
         3.000e+01],
        ...,
        [4.000e+00, 2.200e+01, 9.100e+01, ..., 0.000e+00, 8.050e+02,
         2.500e+01],
        [4.000e+00, 2.700e+01, 8.500e+01, ..., 1.000e+00, 1.470e+02,
         2.500e+01],
        [4.000e+00, 2.200e+01, 6.900e+01, ..., 0.000e+00, 8.830e+02,
         2.500e+01]])

In [14]:
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.1, random_state=0)

In [15]:
from sklearn.model_selection import RandomizedSearchCV
criterion=['mse','mae']
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1200, num = 50)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'criterion':criterion,
                'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


0

In [16]:
from sklearn.ensemble import RandomForestRegressor


In [17]:
clf= RandomForestRegressor()

In [18]:
clf_random = RandomizedSearchCV(estimator=clf, param_distributions=random_grid,
                              n_iter = 10,  
                              cv = 3, verbose=2, random_state=42, n_jobs=1)


In [20]:
from sklearn.ensemble import RandomForestRegressor
from pyecharts.charts.basic_charts import bar

#{'n_estimators': 506, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 70, 'criterion': 'mse', 'bootstrap': True}
rf=RandomForestRegressor(criterion='mse',bootstrap=True,max_features='sqrt', max_depth=70,min_samples_split=10, n_estimators=506,min_samples_leaf=4)

rf.fit(X_train, Y_train) 
Y_train_pred=rf.predict(X_train)
Y_test_pred=rf.predict(X_test)
#变量重要性
print ("变量重要性：",rf.feature_importances_)
feature=['所在区','户型','面积','装修','有无电梯','小区名称','所在位置']




变量重要性： [0.40291331 0.05589718 0.08373913 0.02904377 0.0814192  0.0917413
 0.2552461 ]


In [91]:
ak='gmWGXazzhusjfBhAXCWX99hRsuf681GC'

In [7]:
import requests
def getlng(address):
    """
    @ address: 名称字符串
    @ 返回值：经度，纬度
    """
    address='山东省威海市'+address
    base_url = "http://api.map.baidu.com/geocoder?address={address}&output=json&key=gmWGXazzhusjfBhAXCWX99hRsuf681GC".format(address=address)
    response = requests.get(base_url)
    answer = response.json()
    latitude = answer['result']['location']['lng']
    longitude = answer['result']['location']['lat']

    return [latitude, longitude]

In [8]:
geocodeB('')

[116.413384, 39.910925]

In [9]:
df['loc']=None

In [10]:
df

,house_id,xiaoqu,district,total_price,unit_price,house_type,floor,area,house_struct,in_area,...,last_tradetime,house_use,house_time,owner_attribute,mortage,property_status,villa_type,weizhi,overall_floor,loc
0,103108554235,富山小区,乳山市,358000.0,4023.0,2室1厅1厨1卫,高楼层,89.00,平层,暂无数据,...,暂无数据,普通住宅,暂无数据,非共有,无抵押,未上传房本照片,暂无数据,乳山市,7.0,None
1,103108538149,海洋明珠,乳山市,108000.0,3244.0,1室0厅1厨1卫,低楼层,33.30,平层,暂无数据,...,暂无数据,普通住宅,暂无数据,非共有,无抵押,未上传房本照片,暂无数据,银滩旅游度假区,26.0,None
2,103105657894,龙海阳光小区,乳山市,548000.0,4529.0,3室2厅1厨1卫,低楼层,121.00,平层,暂无数据,...,暂无数据,普通住宅,暂无数据,非共有,无抵押,未上传房本照片,暂无数据,银滩旅游度假区,6.0,None
3,103105816942,爱琴海景西区,乳山市,168000.0,4800.0,1室1厅1厨1卫,低楼层,35.00,平层,暂无数据,...,暂无数据,普通住宅,暂无数据,非共有,无抵押,未上传房本照片,暂无数据,银滩旅游度假区,19.0,None
4,103105658048,海韵豪庭,乳山市,315000.0,4375.0,2室1厅1厨1卫,低楼层,72.00,平层,暂无数据,...,暂无数据,普通住宅,暂无数据,非共有,无抵押,未上传房本照片,暂无数据,银滩旅游度假区,6.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14391,103105474125,凤凰湖B区,荣成市,330000.0,4384.0,2室2厅1厨1卫,低楼层,75.29,平层,暂无数据,...,暂无数据,普通住宅,暂无数据,共有,无抵押,未上传房本照片,暂无数据,石岛,6.0,None
14392,103105464290,亲海花园,荣成市,520000.0,4643.0,3室1厅1厨1卫,低楼层,112.00,平层,暂无数据,...,暂无数据,普通住宅,暂无数据,非共有,无抵押,未上传房本照片,暂无数据,石岛,7.0,None
14393,103106765383,瀛海美地,荣成市,380000.0,4176.0,2室1厅1厨1卫,高楼层,91.00,平层,暂无数据,...,暂无数据,普通住宅,暂无数据,共有,无抵押,未上传房本照片,暂无数据,石岛,6.0,None
14394,103106752170,凤凰湖D区,荣成市,300000.0,3530.0,2室2厅1厨1卫,高楼层,85.00,平层,暂无数据,...,暂无数据,普通住宅,暂无数据,共有,无抵押,未上传房本照片,暂无数据,石岛,7.0,None


In [ ]:
df['loc']=df.apply(lambda x:geocodeB(x.district+x.weizhi),axis=1)

In [ ]:
df

In [22]:
rf.feature_importances_.tolist()

[0.4029133134280569,
 0.055897184986760405,
 0.08373912840593176,
 0.029043765279319123,
 0.08141920058948082,
 0.09174130474881993,
 0.25524610256163094]

In [37]:
app = JupyterDash('Hello Dash Style')
colors = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c574b']
t=np.random.randn(20000)
app.layout = html.Div(
   [
       html.Div(
            id='pic26',
            style={'width': '80%', 'height': '200%', 'padding-left': '10%', 'padding-right': '10%',
                   'float': 'left', 'align-items': 'center', 'justify-content': 'center', 'background-color': '#111111'},
            children=[
                dcc.Graph(
                    id='importance',
                    style={'height': '70vh'},
                    figure=dict(
                        data=[go.Bar(
                            x=['所在区','户型','面积','装修','有无电梯','小区名称','所在位置'],
                            y=rf.feature_importances_,
                            marker=dict(colorscale='Viridis', color=t,
                                        showscale=True),
                            textposition='auto',
                            opacity=0.7
                        )],

                        layout=go.Layout(title='各变量重要性',
                                         paper_bgcolor="#111111",
                                         plot_bgcolor='#111111',
                                         font=dict(
                                             family="Times New Roman", size=20, color='#7fdbff'),
                                         yaxis=dict(title='重要性指数',
                                                    titlefont=dict(
                                                        color='rgb(148, 103, 189)', size=24),
                                                    tickfont=dict(
                                                        color='#7FDBFF', size=24,),
                                                    tickwidth=4,
                                                    tickcolor='#7FDBFF',
                                                    showline=True,
                                                    linecolor='#7FDBFF',
                                                    linewidth=2,
                                                    showticklabels=True,
                                                    autorange=True,
                                                    type='linear',
                                                    ),
                                         xaxis=dict(title='变量',
                                                    titlefont=dict(
                                                        color='rgb(148, 103, 189)', size=24),
                                                    showline=True,
                                                    linecolor='#7FDBFF',
                                                    linewidth=2,
                                                    autorange=True
                                                    )

                                         )

                    )
                )
                
            ]

        )
   ]
)



    

In [38]:
app

In [31]:
type(df['district'].sort_values().unique())

numpy.ndarray

In [30]:
type(df['district'].sort_values().value_counts())

pandas.core.series.Series

In [36]:
rf.feature_importances_

array([0.40291331, 0.05589718, 0.08373913, 0.02904377, 0.0814192 ,
       0.0917413 , 0.2552461 ])